1) Lea el archivo csv “marketing_digital.csv” y cárguelo en un Dataframe llamado data, imprima
los primeros 5 registros para revisar que se haya cargado de manera correcta.

In [5]:
# 1) Leer el archivo csv “marketing_digital.csv” y cargarlo en un DataFrame llamado data
import pandas as pd

csv_path = "/content/marketing_digital.csv"
data = pd.read_csv(csv_path)

data.head()


,Edad,Ingreso_Mensual,Frecuencia_Compra,Tiempo_Web_Min,Descuento_Usado,Campanas_Vistas,Compra
0,65,28501.864093,12,58.477738,1,4,0
1,22,27821.725056,19,14.499604,1,0,0
2,43,21533.399897,10,15.565207,0,1,0
3,21,10112.272028,15,41.490982,0,7,0
4,37,14015.346819,2,34.183489,1,0,0


2) Calcule la proporción de cada clase de la variable objetivo

In [6]:
# 2) Calcular la proporción de cada clase de la variable objetivo (Compra)
conteos = data["Compra"].value_counts()
proporciones = data["Compra"].value_counts(normalize=True)

print("Conteos por clase (Compra):")
print(conteos)
print("\nProporciones por clase (Compra):")
print(proporciones)


Conteos por clase (Compra):
Compra
0    522
1    428
Name: count, dtype: int64

Proporciones por clase (Compra):
Compra
0    0.549474
1    0.450526
Name: proportion, dtype: float64


3) Defina variables predictoras y la variable objetivo.

In [7]:
# 3) Definir variables predictoras (X) y la variable objetivo (y)
X = data.drop(columns=["Compra"])
y = data["Compra"]

print("X shape:", X.shape)
print("y shape:", y.shape)
print("\nColumnas predictoras:", list(X.columns))


X shape: (950, 6)
y shape: (950,)

Columnas predictoras: ['Edad', 'Ingreso_Mensual', 'Frecuencia_Compra', 'Tiempo_Web_Min', 'Descuento_Usado', 'Campanas_Vistas']


4) Escale las variables predictoras para estandarizarlas

In [8]:
# 4) Escalar las variables predictoras para estandarizarlas
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

X_scaled.head()


,Edad,Ingreso_Mensual,Frecuencia_Compra,Tiempo_Web_Min,Descuento_Usado,Campanas_Vistas
0,1.397058,0.445656,0.316334,1.535099,1.025590,-0.169746
1,-1.424130,0.359064,1.608482,-1.387631,1.025590,-1.548026
2,-0.046341,-0.441531,-0.052852,-1.316812,-0.975048,-1.203456
3,-1.489739,-1.895608,0.870112,0.406181,-0.975048,0.863964
4,-0.439995,-1.398689,-1.529593,-0.079465,1.025590,-1.548026


5) Construya el modelo KNN (use 5 vecinos), SVC y el de árboles de clasificación

In [9]:
# 5) Construir el modelo KNN (5 vecinos), SVC y Árbol de Clasificación
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

models_k5 = {
    "KNN (k=5)": Pipeline([("scaler", StandardScaler()),
                          ("model", KNeighborsClassifier(n_neighbors=5))]),
    "SVC": Pipeline([("scaler", StandardScaler()),
                     ("model", SVC())]),
    "Árbol": DecisionTreeClassifier(random_state=42)
}

models_k5


{'KNN (k=5)': Pipeline(steps=[('scaler', StandardScaler()),
                 ('model', KNeighborsClassifier())]),
 'SVC': Pipeline(steps=[('scaler', StandardScaler()), ('model', SVC())]),
 'Árbol': DecisionTreeClassifier(random_state=42)}

6) Aplique validación cruzada con 5 folds

In [10]:
# 6) Aplicar validación cruzada con 5 folds
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv

StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

7) Obtenga accuracy para los 3 modelos

In [11]:
# 7) Obtener accuracy para los 3 modelos (CV 5 folds)
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

acc_results = {}
for name, model in models_k5.items():
    scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy")
    acc_results[name] = {"accuracy_mean": scores.mean(), "accuracy_std": scores.std()}

acc_df = pd.DataFrame(acc_results).T.sort_values("accuracy_mean", ascending=False)
acc_df


,accuracy_mean,accuracy_std
SVC,0.711579,0.020357
KNN (k=5),0.686316,0.022723
Árbol,0.618947,0.012276


8) Obtenga f1 para los 3 modelos

In [12]:
# 8) Obtener f1 para los 3 modelos (CV 5 folds)
f1_results = {}
for name, model in models_k5.items():
    scores = cross_val_score(model, X, y, cv=cv, scoring="f1")
    f1_results[name] = {"f1_mean": scores.mean(), "f1_std": scores.std()}

f1_df = pd.DataFrame(f1_results).T.sort_values("f1_mean", ascending=False)
f1_df

,f1_mean,f1_std
SVC,0.668315,0.022166
KNN (k=5),0.645988,0.026055
Árbol,0.575769,0.019718


9) Repita el proceso, pero ahora utilice 9 vecinos. ¿Hay alguna variación en el accuracy o f1 al
hacer este cambio?

In [13]:
# 9) Repetir el proceso con 9 vecinos. Comparar accuracy y f1 vs k=5
models_k9 = {
    "KNN (k=9)": Pipeline([("scaler", StandardScaler()),
                          ("model", KNeighborsClassifier(n_neighbors=9))]),
    "SVC": models_k5["SVC"],
    "Árbol": models_k5["Árbol"]}

def eval_cv(models, scoring):
    out = {}
    for name, model in models.items():
        s = cross_val_score(model, X, y, cv=cv, scoring=scoring)
        out[name] = {"mean": s.mean(), "std": s.std()}
    return pd.DataFrame(out).T.sort_values("mean", ascending=False)

acc_k5 = eval_cv(models_k5, "accuracy").rename(columns={"mean":"accuracy_mean","std":"accuracy_std"})
f1_k5  = eval_cv(models_k5, "f1").rename(columns={"mean":"f1_mean","std":"f1_std"})

acc_k9 = eval_cv(models_k9, "accuracy").rename(columns={"mean":"accuracy_mean","std":"accuracy_std"})
f1_k9  = eval_cv(models_k9, "f1").rename(columns={"mean":"f1_mean","std":"f1_std"})

print("=== Accuracy (k=5) ===")
display(acc_k5)
print("\n=== Accuracy (k=9) ===")
display(acc_k9)

print("\n=== F1 (k=5) ===")
display(f1_k5)
print("\n=== F1 (k=9) ===")
display(f1_k9)

# Variación específica en KNN
delta_acc = acc_k9.loc["KNN (k=9)","accuracy_mean"] - acc_k5.loc["KNN (k=5)","accuracy_mean"]
delta_f1  = f1_k9.loc["KNN (k=9)","f1_mean"] - f1_k5.loc["KNN (k=5)","f1_mean"]


=== Accuracy (k=5) ===


,accuracy_mean,accuracy_std
SVC,0.711579,0.020357
KNN (k=5),0.686316,0.022723
Árbol,0.618947,0.012276



=== Accuracy (k=9) ===


,accuracy_mean,accuracy_std
SVC,0.711579,0.020357
KNN (k=9),0.709474,0.014661
Árbol,0.618947,0.012276



=== F1 (k=5) ===


,f1_mean,f1_std
SVC,0.668315,0.022166
KNN (k=5),0.645988,0.026055
Árbol,0.575769,0.019718



=== F1 (k=9) ===


,f1_mean,f1_std
SVC,0.668315,0.022166
KNN (k=9),0.667585,0.011321
Árbol,0.575769,0.019718



Variación KNN: Δaccuracy = +0.0232, Δf1 = +0.0216


10) ¿Cuál de los 3 algoritmos funciona mejora para la base de datos proporcionada? Justifique
su respuesta.

In [14]:
# 10) ¿Cuál algoritmo funciona mejor? (basado en CV, usando k=9 para KNN)
summary = pd.concat(
    [
        acc_k9[["accuracy_mean","accuracy_std"]],
        f1_k9[["f1_mean","f1_std"]]
    ],
    axis=1
).sort_values(["f1_mean","accuracy_mean"], ascending=False)

best = summary.index[0]
print("Resumen de desempeño (CV 5 folds):")
display(summary)

print(f"\nMejor modelo según F1 (y desempate por accuracy): {best}")

Resumen de desempeño (CV 5 folds):


,accuracy_mean,accuracy_std,f1_mean,f1_std
SVC,0.711579,0.020357,0.668315,0.022166
KNN (k=9),0.709474,0.014661,0.667585,0.011321
Árbol,0.618947,0.012276,0.575769,0.019718



Mejor modelo según F1 (y desempate por accuracy): SVC
Justificación: SVC logra el mayor F1 promedio y también el mayor (o casi el mayor) accuracy promedio.
